In [45]:
# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import MobileNetV2
import matplotlib.pyplot as plt

In [56]:
#data sugmentation
# Preprocessing the Training set
datagen = ImageDataGenerator(rescale=1./255,validation_split=0.30,
                            rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')
training_set = datagen.flow_from_directory('Dataset',
                                                    target_size=(224,224),
                                                    color_mode="rgb",
                                                    subset="training",
                                                    class_mode="binary",
                                                    batch_size=32,
                                                    shuffle=True)

Found 21 images belonging to 3 classes.


In [57]:
valid_set=datagen.flow_from_directory(directory="Dataset",
                                    target_size=(224,224),
                                    color_mode="rgb",
                                    subset="validation",
                                    class_mode="binary",
                                    batch_size=32,
                                    shuffle=True)

Found 9 images belonging to 3 classes.


In [58]:
from glob import glob
folders = glob('Dataset/*')
len(folders)

3

In [50]:
# 
# def plotImages(images_arr):
#     fig, axes = plt.subplots(1, 5, figsize=(20, 20))
#     axes = axes.flatten()
#     for img, ax in zip(images_arr, axes):
#         ax.imshow(img)
#     plt.tight_layout()
#     plt.show()

In [51]:
# images = [training_set[0][0][0] for i in range(5)]
# plotImages(images)

In [53]:
IMAGE_SIZE = [224, 224]
mobilenet = MobileNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# don't train existing weights
for layer in mobilenet.layers:
  layer.trainable = False

# our layers - you can add more if you want
x = Flatten()(mobilenet.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=mobilenet.input, outputs=prediction)

In [60]:
# tell the model what cost and optimization method to use
model.compile( loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'] )

In [62]:
# fit the model
r = model.fit( training_set,
  validation_data=valid_set,
  epochs=5,
    batch_size=16
)

Epoch 1/5
1/1 [==============================] - 3s 3s/step - loss: 0.6529 - accuracy: 0.9048 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/5
1/1 [==============================] - 3s 3s/step - loss: 1.7598e-07 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 3s 3s/step - loss: 0.0108 - accuracy: 1.0000 - val_loss: 8.8744e-07 - val_accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 2s 2s/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 5.1656e-06 - val_accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 2s 2s/step - loss: 4.3168e-05 - accuracy: 1.0000 - val_loss: 2.2581e-05 - val_accuracy: 1.0000


In [78]:
pred1=np.argmax(model.predict(valid_set,verbose=1),axis=1)

1/1 [==============================] - 2s 2s/step


In [87]:
valid_set.class_indices

{'modi': 0, 'pratyusa': 1, 'trump': 2}

In [98]:
from tensorflow.keras.models import load_model
model.save('my_model.h5')

INFO:tensorflow:Assets written to: my_model.json\assets


c:\python37\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
c:\python37\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [93]:
from tensorflow.keras.models import load_model
model_att = load_model('my_model.h5')

In [97]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('pratyusa.jpg', target_size = (224,224))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = model_att.predict(test_image)
result = np.argmax(result)
result
if result==0:
    print("Modi")
elif result == 1:
    print('Pratyusa')
elif result == 2:
    print('Trump')

Modi
